In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import numpy as np
from glob import glob
from PIL import Image
import os

Options

In [12]:
opt={#BASE OPTIONS
    "dataroot":"./datasets/prostate_segmentation/",
    'phase':'train',
    'image_size':256
}
class Map(dict):
    """
    Example:
    m = Map({'first_name': 'Eduardo'}, last_name='Pool', age=24, sports=['Soccer'])
    """
    def __init__(self, *args, **kwargs):
        super(Map, self).__init__(*args, **kwargs)
        for arg in args:
            if isinstance(arg, dict):
                for k, v in arg.items():
                    self[k] = v

        if kwargs:
            for k, v in kwargs.items():
                self[k] = v

    def __getattr__(self, attr):
        return self.get(attr)

    def __setattr__(self, key, value):
        self.__setitem__(key, value)

    def __setitem__(self, key, value):
        super(Map, self).__setitem__(key, value)
        self.__dict__.update({key: value})

    def __delattr__(self, item):
        self.__delitem__(item)

    def __delitem__(self, key):
        super(Map, self).__delitem__(key)
        del self.__dict__[key]
opt = Map(opt)

make dataset

In [34]:
class AlignedDataset(Dataset):
    def __init__(self,opt):
        #dataroot
        self.opt = opt
        self.root = opt.dataroot
        self.dir = os.path.join(opt.dataroot,'train')
        self.picPaths = sorted(glob(self.dir+'/*'))
        self.name = 'Aligned'
    def __getitem__(self,index):
        ab_path = self.picPaths[index]
        ab = Image.open(ab_path).convert('RGB')
        side = self.opt.image_size
        A = ab.crop((0,0,side,side)).resize((self.opt.image_size,self.opt.image_size),Image.BICUBIC)
        B = ab.crop((side,0,side,side)).resize((self.opt.image_size,self.opt.image_size),Image.BICUBIC)
        
        A = transforms.ToTensor()(A)
        B = transforms.ToTensor()(B)

        A = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(A)
        B = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(B)
        
        return {'A':A,'B':B,
                'A_paths': ab_path, 'B_paths': ab_path}
        
    def __len__(self):
        return len(self.picPaths)

In [35]:
dataset = AlignedDataset(opt)

In [51]:
class uNet(nn.Module):
    def __init__(self,opt):
        self.image_size = opt.image_size
        self.layers = {}
        while self.image_size > 1:
            key = 'conv_'+str(self.image_size)
            self.layers[key] = nn.Conv2d(in_channels = self.image_size,out_channels= int(self.image_size/2),stride=2,kernel_size=4)
            self.image_size=int(self.image_size /2)
        self.layers['downRelu'] = nn.LeakyReLU(negative_slope=0.2,True)
        self.layers['norm'] = nn.BatchNorm2d(negative_slope=0.2,True)


In [52]:
unet = uNet(opt)

128
64
32
16
8
4
2
1


{'conv_256': Conv2d(256, 128, kernel_size=(4, 4), stride=(2, 2)),
 'conv_128': Conv2d(128, 64, kernel_size=(4, 4), stride=(2, 2)),
 'conv_64': Conv2d(64, 32, kernel_size=(4, 4), stride=(2, 2)),
 'conv_32': Conv2d(32, 16, kernel_size=(4, 4), stride=(2, 2)),
 'conv_16': Conv2d(16, 8, kernel_size=(4, 4), stride=(2, 2)),
 'conv_8': Conv2d(8, 4, kernel_size=(4, 4), stride=(2, 2)),
 'conv_4': Conv2d(4, 2, kernel_size=(4, 4), stride=(2, 2)),
 'conv_2': Conv2d(2, 1, kernel_size=(4, 4), stride=(2, 2))}